In [3]:
import pandas as pd
import numpy as np

# Using pandas to load the CSV file into a DataFrame.
# Inspecting the first few rows to understand the structure and content.
df = pd.read_csv('stop_tb_data2.csv')
print(df.head())



  country_name  year iso3_code  fragility  fragilityindex g_whoregion  \
0       Zambia  2000       ZMB  Neighbour            87.2         AFR   
1       Zambia  2001       ZMB  Neighbour            87.2         AFR   
2       Zambia  2002       ZMB  Neighbour            87.2         AFR   
3       Zambia  2003       ZMB  Neighbour            87.2         AFR   
4       Zambia  2004       ZMB  Neighbour            87.2         AFR   

   gendertool gf_eligible_2016 gf_eligible_2018    globalplangroup  ...  \
0         NaN              Yes              Yes  High TB-HIV, SADC  ...   
1         NaN              Yes              Yes  High TB-HIV, SADC  ...   
2         NaN              Yes              Yes  High TB-HIV, SADC  ...   
3         NaN              Yes              Yes  High TB-HIV, SADC  ...   
4         NaN              Yes              Yes  High TB-HIV, SADC  ...   

   used_2021_defs_flg xdr xdr_coh xdr_died  xdr_fail xdr_lost  xdr_succ xpert  \
0                 NaN NaN    

In [4]:
# Checking the number of missing values per column
missing_values_count = df.isnull().sum()

# Checking the percentage of missing values per column
missing_values_percentage = df.isnull().mean() * 100

# Print results
print(missing_values_count)
print(missing_values_percentage)


country_name       0
year               0
iso3_code          0
fragility          0
fragilityindex     0
                  ..
xdr_lost          20
xdr_succ          20
xpert             23
xpert_eqa         23
xpert_etrans      23
Length: 934, dtype: int64
country_name       0.000000
year               0.000000
iso3_code          0.000000
fragility          0.000000
fragilityindex     0.000000
                    ...    
xdr_lost          71.428571
xdr_succ          71.428571
xpert             82.142857
xpert_eqa         82.142857
xpert_etrans      82.142857
Length: 934, dtype: float64


In [6]:
# Dropping columns with more than 50% missing values


df = df.dropna(thresh=len(df) * 0.5, axis=1)

# Filling missing values with median for numerical columns
numeric_columns = df.select_dtypes(include=[np.number]).columns
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())


NameError: name 'np' is not defined